In [1]:
%%time
import warnings
import numpy
import pandas as pd
import onnxruntime as rt
from tqdm import tqdm
from sklearn.compose import ColumnTransformer
from sklearn.datasets import load_iris
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
try:
    from sklearn.ensemble import HistGradientBoostingClassifier
except ImportError:
    HistGradientBoostingClassifier = None
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from skl2onnx.common.data_types import FloatTensorType, StringTensorType
from skl2onnx import to_onnx, update_registered_converter
from skl2onnx.sklapi import CastTransformer, ReplaceTransformer
from skl2onnx.common.shape_calculator import (
    calculate_linear_classifier_output_shapes)
from onnxmltools.convert.xgboost.operator_converters.XGBoost import (
    convert_xgboost)
from onnxmltools.convert.lightgbm.operator_converters.LightGbm import (
    convert_lightgbm)


Wall time: 2.99 s


In [3]:
df = pd.read_csv('C:/Users/bodie/Documents/Spreadsheets/emails_not_normalized.csv')

In [8]:
df.info

<bound method DataFrame.info of       Unnamed: 0  dir_name  is_spam  \
0              0      spam        1   
1              1      spam        1   
2              2      spam        1   
3              3      spam        1   
4              4      spam        1   
...          ...       ...      ...   
9348        9348  hard_ham        0   
9349        9349  hard_ham        0   
9350        9350  hard_ham        0   
9351        9351  hard_ham        0   
9352        9352  hard_ham        0   

                                             from                 domain  \
0                             12a1mailbot1@web.de                 web.de   
1          "slim down" <taylor@s3.serveimage.com>      s3.serveimage.com   
2           "slim down" <sabrina@mx3.1premio.com>        mx3.1premio.com   
3             account services <wsup@playful.com>            playful.com   
4          "slim n trim" <yenene@mx2.1premio.com>        mx2.1premio.com   
...                                        

In [9]:
%%time
if 'is_spam' in df:
    y = df['is_spam'].values
    del df['is_spam']
    X = df['full_text'].str.lower()

Wall time: 92.6 ms


In [10]:
X

0       from 12a1mailbot1@web.de thu aug 22 13:17:22 2...
1       from ilug-admin@linux.ie thu aug 22 13:27:39 2...
2       from sabrina@mx3.1premio.com thu aug 22 14:44:...
3       from wsup@playful.com thu aug 22 16:17:00 2002...
4       from social-admin@linux.ie thu aug 22 16:37:34...
                              ...                        
9348    return-path: ler@lerami.lerctr.org delivery-da...
9349    from bounce-ora_webprog-1083425@newsletter.ore...
9350    return-path: linux-announce-recipients-owner-m...
9351    return-path:  received: from usw-sf-list2.sour...
9352    mv 00001.7c7d6921e671bbe18ebb5f893cd9bb35 0000...
Name: full_text, Length: 9353, dtype: object

In [13]:
%%time
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

text_clf = Pipeline([
  ('vect', CountVectorizer()),
   ('tfidf', TfidfTransformer()),
('clf', MultinomialNB())])

Wall time: 0 ns


In [14]:
%%time
X_train, X_test, y_train, y_test = train_test_split(X, y)
text_clf.fit(X_train, y_train)

Wall time: 2.56 s


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [15]:
%%time
onx = to_onnx(text_clf, X_train.values,
              target_opset=12)

C:\Users\bodie\Anaconda3\lib\site-packages\skl2onnx\operator_converters\text_vectoriser.py:190: UserWarning: Converter for TfidfVectorizer will use scikit-learn regular expression by default in version 1.6.
  UserWarning)
C:\Users\bodie\Anaconda3\lib\site-packages\skl2onnx\common\_container.py:698: UserWarning: Unable to find operator 'Tokenizer' in domain 'com.microsoft' in ONNX, op_version is forced to 1.
  op_type, domain))


Wall time: 772 ms


In [16]:
%%time
with open("text_model.onnx", "wb") as f:
    f.write(onx.SerializeToString())

Wall time: 12.4 ms


In [17]:
%%time
from mlprodict.onnxrt import OnnxInference
from onnxruntime import InferenceSession

sess = InferenceSession(onx.SerializeToString())
pred_ort = sess.run(None, {'X': X_test.values})[0]

pred_skl = text_clf.predict(X_test)

print("Onnx Runtime prediction:\n", pred_ort[:5])
print("Sklearn rediction:\n", pred_skl[:5])

Onnx Runtime prediction:
 [0 0 0 0 0]
Sklearn rediction:
 [0 0 0 0 0]
Wall time: 3.18 s
